In [2]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [3]:
week = 21 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

13 March 2023 ; 12:47:21


In [4]:
features = ['PTS','VORP','FG','WS','MP','TRB','AST','PER','BPM']#, 'GS'] ### GS
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 8
# Propotion of maximum games played
gp = 1/4

In [5]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/roy/roy_data.csv')

# Load predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') # Load up-to-date data
data = data.groupby('roy').get_group(1)

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,A.J. Green,SG,23,MIL,30,0,9.7,1.6,3.6,0.444,...,3.96,115.25,111.40,3.84,3.88,115.31,111.52,3.79,1,1
11,MarJon Beauchamp,SF,22,MIL,41,9,14.0,2.0,5.0,0.390,...,3.96,115.25,111.40,3.84,3.88,115.31,111.52,3.79,1,1
17,A.J. Lawson,SG,22,DAL,13,0,4.8,1.0,2.0,0.500,...,0.68,117.78,117.12,0.66,0.68,117.71,117.06,0.64,1,1
25,Jaden Hardy,SG,20,DAL,34,1,12.5,2.4,5.6,0.434,...,0.68,117.78,117.12,0.66,0.68,117.71,117.06,0.64,1,1
38,AJ Griffin,SF,19,ATL,60,11,19.7,3.5,7.4,0.467,...,0.04,116.63,116.59,0.04,0.06,116.83,116.77,0.06,1,1


In [6]:
# Set training data
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] > season_progress * gp] # Minimum games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [7]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player', 'MP' ,'Tm', 'PER', 'VORP', 'WS', 'FG', 'PTS', 'Seed', 'Share']]
res.head(5)

,Player,MP,Tm,PER,VORP,WS,FG,PTS,Seed,Share
0,Paolo Banchero,33.8,ORL,15.0,0.268852,2.688525,6.6,20.0,13,0.256398
1,Walker Kessler,22.3,UTA,21.4,2.178125,7.559375,3.8,8.6,9,0.197070
2,Jalen Duren,24.9,DET,16.7,0.455556,5.466667,3.6,8.6,15,0.103228
3,Jaden Ivey,30.2,DET,12.1,-0.661290,0.132258,5.2,15.4,15,0.096486
4,Bennedict Mathurin,27.8,IND,13.4,-0.979104,2.080597,5.3,16.6,11,0.078698


In [8]:
res.to_csv(path + f'Results/roy/results_week_{week}.csv', index = None)